## Train Results

In [1]:
import pandas as pd

# Load the CSV file (replace 'your_file.csv' with the actual path)
train_results_path = './results/240364_train_fusion_rates_results.csv'
train_id = train_results_path.split('/')[-1].split('_')[0]
print(f"Processing train id: {train_id}")

df = pd.read_csv(train_results_path)

# Clean the 'subject_id' column
df['subject_id'] = df['subject_id'].str.strip("[]").str.replace("'", "", regex=False)

# drop trial_id column
df = df.drop(columns=['trial_id'])

# Convert rate columns to numeric, coercing errors to NaN
rate_cols = ['smartwatch_rate', 'video_rate', 'gt_rate']
df[rate_cols] = df[rate_cols].apply(pd.to_numeric, errors='coerce')

# Drop rows with invalid subject_ids (e.g., '...' or empty)
print(df['subject_id'].unique())
# Compute subject-wise average for each rate
subject_avg = df.groupby('subject_id')[rate_cols].mean().reset_index()

# drop subject_id P10
subject_avg = subject_avg[subject_avg['subject_id'] != 'P10']

# drop index column
subject_avg = subject_avg.reset_index(drop=True)
# Print or save the result

print(subject_avg)

subject_avg.to_csv(f'./results/{train_id}_train_subject_wise_avg_rates.csv', index=False)  # Optional: save to CSV


Processing train id: 240364
['wa1' 'ng5' 'ng8' 'ng3' 'P14' 'ng4' 'ms2' 'P20' 'P5' 'P0' 'P12' 'ng6'
 'P21' 'P16' 'ng1' 'P11' 'P3']
   subject_id  smartwatch_rate  video_rate     gt_rate
0          P0       183.000000   63.000000  228.000000
1         P11       120.000000   54.000000  114.000000
2         P12              NaN         NaN         NaN
3         P14       103.800000   51.600000   63.600000
4         P16       102.000000   78.000000  100.363636
5         P20       102.000000   36.000000  156.000000
6         P21       108.381818   67.109091  112.690909
7          P3       123.000000   45.000000  189.000000
8          P5       102.000000   30.000000   88.000000
9         ms2       111.000000  105.000000  111.000000
10        ng1              NaN         NaN         NaN
11        ng3       116.000000   70.000000  120.000000
12        ng4       112.000000  102.000000  118.000000
13        ng5              NaN         NaN         NaN
14        ng6        99.000000   66.000000   

## Test Result

In [ ]:
import pandas as pd
import numpy as np

# Load the CSV file (replace with your actual path if needed)
test_results_path = './results/fusion_test_default/test_results_414469_smartwatch_model_test_fusion_rmse_per_batch.csv'
test_id = test_results_path.split('/')[-1].split('_')[0]
print(f"Processing test id: {test_id}")

test_result_folder = test_results_path.split('/')[-2]

df = pd.read_csv(test_results_path)

# Clean the 'subject_id' column
df['subject_id'] = df['subject_id'].str.strip("[]").str.replace("'", "", regex=False)

# Drop the trial_id column
df = df.drop(columns=['trial_id'])

# Ensure rate columns are numeric
# subject_id,trial_id,smartwatch_rate,video_rate,fused_rate,gt_rate,rmse_cpm,smartwatch_depth,video_depth,fused_depth,gt_depth,rmse_depth

rate_cols = ['rmse_cpm','smartwatch_rate', 'video_rate', 'gt_rate', 'smartwatch_depth', 'video_depth', 'fused_depth', 'gt_depth', 'rmse_depth']
df[rate_cols] = df[rate_cols].apply(pd.to_numeric, errors='coerce')

# Drop any rows with invalid or empty subject_ids
df = df[df['subject_id'].notna() & (df['subject_id'] != '')]

# Compute subject-wise average of the provided rmse_cpm and mean rates
subject_avg = (
    df
    .groupby('subject_id', as_index=False)
    .agg({
        'rmse_cpm': 'mean',
        'fused_rate': 'mean',  # Assuming fused_rate is the same as rmse_cpm
        'smartwatch_rate': 'mean',
        'video_rate': 'mean',
        'gt_rate': 'mean',
        'smartwatch_depth': 'mean',
        'video_depth': 'mean',
        'fused_depth': 'mean',
        'gt_depth': 'mean',
        'rmse_depth': 'mean'
    })
    .rename(columns={
        'rmse_cpm': 'avg_fused_rmse_cpm',
        'fused_rate': 'avg_fused_rate',  # Assuming fused_rate is the same as rmse_cpm
        'smartwatch_rate': 'avg_smartwatch_rate',
        'video_rate': 'avg_video_rate',
        'gt_rate': 'avg_gt_rate',
        'smartwatch_depth': 'avg_smartwatch_depth',
        'video_depth': 'avg_video_depth',
        'fused_depth': 'avg_fused_depth',
        'gt_depth': 'avg_gt_depth',
        'rmse_depth': 'avg_rmse_depth'
    })
)

# Drop subject_id P10 if present
subject_avg = subject_avg[subject_avg['subject_id'] != 'P10']

# --- New: compute RMSE of smartwatch_rate vs gt_rate, and video_rate vs gt_rate ---

# function to compute rmse between two series
def compute_rmse(a, b):
    return np.sqrt(np.mean((a - b) ** 2))

# group and compute
rmse_metrics = (
    df
    .groupby('subject_id')
    .apply(lambda g: pd.Series({
        'rmse_smartwatch_cpm': compute_rmse(g['smartwatch_rate'], g['gt_rate']),
        'rmse_video_cpm':      compute_rmse(g['video_rate'],     g['gt_rate']),
        'rmse_fused_cpm':      compute_rmse(g['fused_rate'],     g['gt_rate']),
        'rmse_smartwatch_depth': compute_rmse(g['smartwatch_depth'], g['gt_depth']),
        'rmse_video_depth':      compute_rmse(g['video_depth'],     g['gt_depth']),
        'rmse_fused_depth':      compute_rmse(g['fused_depth'],     g['gt_depth']),

    }))
    .reset_index()
)

# merge the new rmse columns into subject_avg
subject_summary = subject_avg.merge(rmse_metrics, on='subject_id')

# Reset index for cleanliness
subject_summary = subject_summary.reset_index(drop=True)

# reorder columns for better readability
subject_summary = subject_summary[['subject_id','avg_gt_rate','avg_fused_rate','rmse_fused_cpm',
                                   'avg_smartwatch_rate', 'rmse_smartwatch_cpm',
                                   'avg_video_rate', 'rmse_video_cpm',
                                      'avg_smartwatch_depth', 'rmse_smartwatch_depth',
                                      'avg_video_depth', 'rmse_video_depth',
                                       'avg_fused_depth', 'rmse_fused_depth','avg_gt_depth',

                                   ]]

# Print and optionally save
print(subject_summary)

output_path = f'./results/{test_result_folder}/final_{test_id}_test_subject_wise_avg_rates.csv'
subject_summary.to_csv(output_path, index=False)
print(f"Saved summary to {output_path}")


Processing test id: test
   subject_id  avg_gt_rate  avg_fused_rate  rmse_fused_cpm  \
0          P0   177.878151      140.879435       46.533808   
1          P1    62.400000       93.131051       30.731051   
2         P11   123.311111      116.987243        7.685049   
3         P12   135.300000      138.230209       27.365196   
4         P14    64.125000       99.805192       35.966629   
5         P15   105.366667      109.425557       17.879042   
6         P16   117.857143      112.204170       13.613236   
7          P2   140.787879      129.362129       15.784125   
8         P20    88.400000       98.623347       10.223347   
9         P21    87.115152       96.661368       19.512391   
10        P22   120.666667      117.222001        3.444666   
11         P3    92.475000      109.224083       18.364543   
12         P4   114.380952      124.987585       22.490517   
13         P5    51.171429       89.960610       39.590897   
14         P7   106.485714      112.689869   

/tmp/ipykernel_71007/2046663567.py:55: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df


# Summary for all subjects

In [1]:
import pandas as pd
import numpy as np
import os

# Load the CSV file (replace with your actual path if needed)
test_results_path = './results/2025-06-18_18-54-57/debug_3817147_fusion_weights_test_fusion_rmse_per_batch.csv'
fname = os.path.basename(test_results_path)
test_id = fname.split('_')[1]  # e.g., "3817147"
print(f"Processing test id: {test_id}")

df = pd.read_csv(test_results_path)

# Clean the 'subject_id' column
df['subject_id'] = df['subject_id'].str.strip("[]").str.replace("'", "", regex=False)

# Drop the trial_id column
df = df.drop(columns=['trial_id'])

# Ensure rate columns are numeric (including fused_rate)
rate_cols = ['fused_rate', 'smartwatch_rate', 'video_rate', 'gt_rate']
df[rate_cols] = df[rate_cols].apply(pd.to_numeric, errors='coerce')

# Drop any rows with invalid or empty subject_ids
df = df[df['subject_id'].notna() & (df['subject_id'] != '')]

# Functions to compute error metrics
def compute_rmse(pred, gt):
    return np.sqrt(np.mean((pred - gt) ** 2))

def compute_mae(pred, gt):
    return np.mean(np.abs(pred - gt))

# --- Per-subject summary (existing) ---
subject_metrics = (
    df
    .groupby('subject_id')
    .apply(lambda g: pd.Series({
        'rmse_fused_cpm':      compute_rmse(g['fused_rate'],     g['gt_rate']),
        'rmse_smartwatch_cpm': compute_rmse(g['smartwatch_rate'], g['gt_rate']),
        'rmse_video_cpm':      compute_rmse(g['video_rate'],     g['gt_rate'])
    }))
    .reset_index()
)

# Save per-subject CSV
subject_output = test_results_path.replace(
    'fusion_weights_test_fusion_rmse_per_batch.csv',
    'subject_wise_rmse.csv'
)
subject_metrics.to_csv(subject_output, index=False)
print(f"Saved per-subject RMSE to {subject_output}")

# --- Overall RMSE (all subjects combined) ---
overall = {
    'rmse_fused_cpm':      compute_rmse(df['fused_rate'],     df['gt_rate']),
    'rmse_smartwatch_cpm': compute_rmse(df['smartwatch_rate'], df['gt_rate']),
    'rmse_video_cpm':      compute_rmse(df['video_rate'],     df['gt_rate'])
}
overall_df = pd.DataFrame([overall])

# Save overall CSV
overall_output = test_results_path.replace(
    'fusion_weights_test_fusion_rmse_per_batch.csv',
    'overall_rmse.csv'
)
overall_df.to_csv(overall_output, index=False)
print(f"Saved overall RMSE to {overall_output}")


Processing test id: 3817147
Saved per-subject RMSE to ./results/2025-06-18_18-54-57/debug_3817147_subject_wise_rmse.csv
Saved overall RMSE to ./results/2025-06-18_18-54-57/debug_3817147_overall_rmse.csv


/tmp/ipykernel_737534/3813468637.py:35: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df
